In [56]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy.sparse import lil_matrix, coo_matrix, csr_matrix, save_npz

In [3]:
table = pd.read_csv('extracted.csv')

In [4]:
user_set = set()
id_set = set()
for _, row in tqdm(table.iterrows(), total=len(table)):
    id_set.add(row['id'])
    user_set.add(row['user_id'])
len(user_set), len(id_set)

100%|██████████| 21207961/21207961 [12:39<00:00, 27929.90it/s]


(172795, 21207961)

In [50]:
A = lil_matrix((len(user_set) + len(id_set), len(user_set) + len(id_set)))
offset_size = len(user_set)
# offset_idx = 3090001

In [11]:
ids = table['id'].values
users = table['user_id'].values
ancestors = table['ancestor_id'].values

In [24]:
temp = []
for ancestor in ancestors:
    try:
        temp.append(int(ancestor))
    except:
        temp.append(None)
len(temp)

21207961

In [51]:
user_id_2_idx = dict()
idx_2_user_id = dict()
id_2_idx = dict()
idx_2_id = dict()
for i, user_id in enumerate(tqdm(user_set)):
    user_id_2_idx[user_id] = i
    idx_2_user_id[i] = user_id
for i, tid in enumerate(tqdm(ids)):
    id_2_idx[tid] = i
    idx_2_id[i] = tid

100%|██████████| 21207961/21207961 [00:11<00:00, 1829589.37it/s]


In [53]:
import pickle
pickle.dump(user_id_2_idx, open("user_id_2_idx.pkl", "wb"))
pickle.dump(idx_2_user_id, open("idx_2_user_id.pkl", "wb"))
pickle.dump(id_2_idx, open("id_2_idx.pkl", "wb"))
pickle.dump(idx_2_id, open("idx_2_id.pkl", "wb"))

In [54]:
for i in tqdm(range(len(ids))):
    # Ancestor ids to node ids and vice versa
    if temp[i] is not None:
        try:
            A[offset_size + id_2_idx[temp[i]], offset_size + id_2_idx[ids[i]]] = 1
            A[offset_size + id_2_idx[ids[i]], offset_size + id_2_idx[temp[i]]] = 1
        except:
            pass
    # User ids to node ids and vice versa
    try:
        A[user_id_2_idx[users[i]], offset_size + id_2_idx[ids[i]]] = 1
        A[offset_size + id_2_idx[ids[i]], user_id_2_idx[users[i]]] = 1
    except:
        pass

100%|██████████| 21207961/21207961 [03:00<00:00, 117741.28it/s]


In [44]:
hatexplain_ids = pd.read_csv("hatexplain.csv")['id'].values
hatexplain_ids

array([13851720, 24198545, 14567516, ...,  9991681,  9992513,  9998729])

In [58]:
save_npz("A_full.npz", A.tocsr())

In [55]:
for hatexplain_id in hatexplain_ids:
    A[offset_size + id_2_idx[hatexplain_id]]